# Set Membership

The cell below defines two **abstract classes**: the first represents a set and basic insert/search operations on it. You will need to impement this API four times, to implement (1) sequential search, (2) binary search tree, (3) balanced search tree, and (4) bloom filter. The second defines the synthetic data generator you will need to implement as part of your experimental framework. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [1]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

# abstract class to represent a set and its insert/search operations
class AbstractSet(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # inserts "element" in the set
    # returns "True" after successful insertion, "False" if the element is already in the set
    # element : str
    # inserted : bool
    @abstractmethod
    def insertElement(self, element):     
        inserted = False
        return inserted   
    
    # checks whether "element" is in the set
    # returns "True" if it is, "False" otherwise
    # element : str
    # found : bool
    @abstractmethod
    def searchElement(self, element):
        found = False
        return found    
    
    
    
# abstract class to represent a synthetic data generator
class AbstractTestDataGenerator(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # creates and returns a list of length "size" of strings
    # size : int
    # data : list<str>
    @abstractmethod
    def generateData(self, size):     
        data = [""]*size
        return data   


Use the cell below to define any auxiliary data structure and python function you may need. Leave the implementation of the main API to the next code cells instead.

In [2]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE

#class Node for implementation of Balanced Search Tree (Left Leaning Red Black Tree)
class Node:

  def __init__(self, key):
      self.key = key
      self.left = None 
      self.right = None 
      self.color = "RED"


  def is_red(self):
      return self.color == "RED"

  def search(self, key):
    while self is not None: 
      if self.key == key:
        return True
      elif key < self.key:
        self = self.left 
      elif key > self.key:
        self = self.right
      else:
        return None

  def rotate_left (self):
    x = self.right
    self.right = x.left
    x.left = self
    x.color = self.color
    self.color = "RED"

    return x

  def rotate_right (self):
    x = self.left
    self.left = x.right
    x.right = self
    x.color = self.color
    self.color = "RED"

    return x

  def flip_colors(self):
    self.color = "RED"
    self.left.color = "BLACK"
    self.right.color = "BLACK"

  def insert(self, element, count):


    key = element

    if key == self.key:
            return False

    if key < self.key:
      if self.left is None:
          self.left = Node(key)
      else:
          self.left.insert(key)
    else:
      if self.right is None:
          self.right = Node(key)
      else:
          self.right.insert(key)
    
  #checks to ensure LLRB properties are respected
    if self.right and self.right.is_red() and (self.left is None or not self.left.is_red()):
      self = self.rotate_left()

    if self.left and self.left.left and self.left.is_red() and self.left.left.is_red():
      self = self.rotate_right()

    if self.left and self.right and self.left.is_red() and self.right.is_red():
      self.flip_colors()

    
    return True

# fnv function used for bloom filter
def fnv1a_32(data):
    data = bytes(data, 'utf-8')
    fnv_offset = 2166136261
    fnv_prime = 16777619
    hash_value = fnv_offset
    for byte in data:
        hash_value ^= byte
        hash_value *= fnv_prime
        hash_value &= 0xFFFFFFFF
    return hash_value

Use the cell below to implement the requested API by means of **sequential search**.

In [3]:
class SequentialSearchSet(AbstractSet):
    
    def __init__(self):
        self.elements = [] #empty list created to hold the empty elements
        
        pass           
    
        
    def insertElement(self, element):
        inserted = False
      
        if element in self.elements:       
          return False

        else:
          self.elements.append(element)
          inserted = True 

        return inserted  


    def searchElement(self, element):     
        found = False

        for e in self.elements:
          if e == element:
            found = True 
            break 

        
        return found    

  

Use the cell below to implement the requested API by means of **binary search tree**.

In [4]:
class BinarySearchTreeSet(AbstractSet):

    class Node:
        def __init__(self, value):
            self.left = None
            self.right = None
            self.value = value
    
    def __init__(self):
        self.root = None
        pass          
     
    def insertElement(self, element):
        inserted = False
        if self.root == None:
            self.root = self.Node(element)
            inserted = True
        else:
            cur = self.root
            while cur is not None:
                if element < cur.value:
                    if cur.left is None:
                        cur.left = self.Node(element)
                        return True
                    else:
                        cur = cur.left
                else:
                    if cur.right is None:
                        cur.right = self.Node(element)
                        return True
                    else:
                        cur = cur.right
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        cur = self.root
        while cur is not None:
            if element == cur:
                found = True
            elif element < cur.value:
                cur = cur.left
            else:
                cur = cur.right 
        return found  

Use the cell below to implement the requested API by means of **balanced search tree**.

In [5]:
class BalancedSearchTreeSet(AbstractSet):
  
  def __init__(self):
      
      self.root = None; 
      
      pass           
    
  
      
  def insertElement(self, element):
    inserted = False

    
    if self.root == None: 
      self.root = Node(element) 
      return True
    inserted = self.root.insert(element) 
  
    
    return inserted
  
  

  def searchElement(self, element): 

      found = False
      if self.root == None:
        return False
      found = self.root.search(element) != None
      
      return found 

Use the cell below to implement the requested API by means of **bloom filter**.

In [6]:
class BloomFilterSet(AbstractSet):
    
    def __init__(self):
        #initializing bit array
        self.bitfield = [False]*958505
        
    def insertElement(self, element):
        inserted = False
        #applying hash functions 7 times
        for i in range(7):
            if i%2==0:
                index = hash(element + str(i)) % len(self.bitfield)
            else:
                index = fnv1a_32(element + str(i)) % len(self.bitfield)
            self.bitfield[index] = True
        inserted = True
        return inserted

    def searchElement(self, element):     
        found = False
        for i in range(7):
            if i%2==0:
                index = hash(element + str(i)) % len(self.bitfield)
            else:
                index = fnv1a_32(element + str(i)) % len(self.bitfield)
            if not self.bitfield[index]:
                return found
        found = True

        return found   

Use the cell below to implement the **synthetic data generator** as part of your experimental framework.

In [7]:
import string
import random

class TestDataGenerator(AbstractTestDataGenerator):
    
    def __init__(self):
        pass           
        
    def generateData(self, size):     
        data = []*size
        for i in range(size):
            length = random.randint(5,20)
            result_str = ''.join(random.choice(string.ascii_letters+string.digits) for i in range(length))
            data += [result_str]
        return data 


Use the cells below for the python code needed to **fully evaluate your implementations**, first on real data and subsequently on synthetic data (i.e., read data from test files / generate synthetic one, instantiate each of the 4 set implementations in turn, then thorouhgly experiment with insert/search operations and measure their performance).

In [8]:
import timeit

# ADD YOUR TEST CODE HERE TO WORK ON REAL DATA


def insertTime(filename, algorithm): 
  file = open(filename, 'r')

  starttime = timeit.default_timer()
  for line in file: 
    for word in line.split():
      algorithm.insertElement(word)
  finaltime = timeit.default_timer() - starttime

  return finaltime 


def searchTime(filename, algorithm): 
  file = open(filename, 'r')

  starttime = timeit.default_timer()
  for line in file: 
    for word in line.split():
      algorithm.searchElement(word)
  finaltime = timeit.default_timer() - starttime

  return finaltime

In [9]:
import timeit
import random
# import matplotlib.pyplot as plt

def data_generator_insert(n):
    n_val = n
    generator = TestDataGenerator()
    insert_words = generator.generateData(n_val)
    return insert_words

def data_generator_search():
    generator = TestDataGenerator()
    search_words = generator.generateData(10)
    return search_words

def insert_time(algorithm : AbstractSet, insert_words : list):
    insert = insert_words
    starttime = timeit.default_timer()
    for word in insert:
        algorithm.insertElement(word)
    endtime = timeit.default_timer()-starttime
    return endtime

def search_time(algorithm : AbstractSet, search_words : list):
    total_time = 0
    search = search_words
    for word in search:
        starttime = timeit.default_timer()
        algorithm.searchElement(word)
        endtime = timeit.default_timer()-starttime
        total_time+=endtime
    return total_time


# INSERT GRAPH CODE
x = []
i = 0
while i < 100001:
    x.append(i)
    i+=10000

y1 = []
y2 = []

for n in x:
    total_time = 0
    total_time_2 = 0
    for i in range(10):
        insert_words = data_generator_insert(n+1)
        insert_words_2 = sorted(insert_words[:])
        word_to_insert = [insert_words[n]]
        n_words = insert_words[:n]
        word_to_insert_2 = [insert_words_2[n]]
        n_words_2 = insert_words_2[:n]
        #choose algorithm of choice
        alg = BalancedSearchTreeSet() 
        alg2 = BalancedSearchTreeSet()
        insert_time(alg, n_words)
        insert_time(alg2, n_words_2)
        total_time +=insert_time(alg, word_to_insert)
        total_time_2 +=insert_time(alg2, word_to_insert_2)
    y1.append(total_time/10)
    y2.append(total_time_2/10)

# code for graph generation
# plt.plot(x, y1, label = 'unsorted', linewidth = 2, marker = '.', markersize = 10, color = 'red')
# plt.plot(x, y2, label = 'sorted', linewidth = 2, marker = '.', markersize = 10, color = 'black')
# plt.title('Balanced Search Tree: Insert')
# plt.xlabel('n number of element in set')
# plt.ylabel('Time Taken to insert (n+1)th element')
# plt.legend()
# plt.show()


#SEARCH GRAPH CODE
x = []
i = 1
while i < 100002:
    x.append(i)
    i+=10000

y1 = []
y2 = []

for n in x:
    total_time = 0
    total_time_2 = 0
    for i in range(10):
        #choose algorithm of choice
        alg = BloomFilterSet()
        alg2 = BloomFilterSet()
        insert_words = data_generator_insert(n)
        insert_time(alg, insert_words)
        insert_time(alg2, insert_words)
        not_in_set = data_generator_search()
        half_set = not_in_set + insert_words[:10]
        word_to_search = [random.choice(not_in_set)]
        word_to_search_2 = [random.choice(half_set)]
        total_time +=search_time(alg, word_to_search)
        total_time_2 +=search_time(alg2, word_to_search_2)
    y1.append(total_time/10)
    y2.append(total_time_2/10)

# Code for graph generation
# plt.plot(x, y1, label = 'element not in set', linewidth = 2, marker = '.', markersize = 10, color = 'green')
# plt.plot(x, y2, label = 'element might or might not be in set', linewidth = 2, marker = '.', markersize = 10, color = 'blue')
# plt.title('Bloom Filter: Search')
# plt.xlabel('n number of elements in set')
# plt.ylabel('Time Taken to search for an element')
# plt.legend()
# plt.show()

